# (一) Autocorrelation

## 1.

In [9]:
import numpy as np
import os
import torch
import torch.nn.functional as F
import time

In [12]:
def autocorrelation(input_seq, m_x, m_y, m_t, L_x=352, L_y=288, L_t=100, device=torch.device('cpu')):
    '''
        input_seq : numpy.ndarray(3d) or torch.Tensor(5d)
        m_x, m_y, m_z : Scalar
        L_x, L_y, L_z : Scalar
    ''' 
    
    if abs(m_x) >= L_x or abs(m_y) >= L_y or abs(m_t) >= L_t:
        return 0
    
    #input_seq = torch.Tensor([[input_seq]]).type(torch.float64).to(device)
    
    if type(input_seq) == np.ndarray and len(input_seq.shape) == 3:
        input_seq = torch.Tensor([[input_seq]]).type(torch.float32).to(device)
    elif type(input_seq) == torch.Tensor and len(input_seq.shape) == 5:
        pass
    else:
        raise ValueError
    
    
    inputs = input_seq[:, :, :L_t-abs(m_t),:L_y-abs(m_y),:L_x-(m_x)]
    filters = input_seq[:, :, -abs(m_t):,-abs(m_y):,-abs(m_x):]
    
    #start_t = time.time()
    
    C_xxx = F.conv3d(inputs, filters).view(-1)[0].cpu().numpy()
    R_xxx = C_xxx / ((L_x-abs(m_x))*(L_y-abs(m_y))*(L_t-abs(m_t)))
    
    #end_t = time.time()
    #print('Autocorrelation value = ',R_xxx,' ; time = ',end_t-start_t)
    return R_xxx

## 2.

In [13]:
yuv_filename = './MOBILE_352x288_10.yuv'

In [14]:
with open(yuv_filename ,'rb') as f:
    width, height, n_frames = 352, 288, 100
    
    Y = []
    U = []
    V = []
    print(n_frames)
    
    for i in range(n_frames):
        yuv_frame = np.frombuffer(f.read(width*height*3//2), dtype=np.uint8)
        Y.append(np.array(yuv_frame[:width*height]).reshape(height, width))
        U.append(np.array(yuv_frame[width*height:-width*height//4]))
        V.append(np.array(yuv_frame[-width*height//4:]))
        
    Y = np.array(Y)
    U = np.array(U)
    V = np.array(V)
    f.close()
    
_Y = []
for frame in Y:
    _Y.append(frame-frame.mean())
    
Y = np.array(_Y)

100


In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')     

R_xxx = np.zeros([21,144,176])

start_t = time.time()

input_seq = torch.Tensor([[Y]]).type(torch.float64).to(device)

for m_x in range(88+1):
    m_x_fill = [m_x]
    if m_x < 88 and m_x !=0:
        m_x_fill.append(176-m_x)
        
    for m_y in range(72+1):
        m_y_fill = [m_y]
        if m_y < 72 and m_y !=0:
            m_y_fill.append(144-m_y)
            
        for m_t in range(11):
            m_t_fill = [10+m_t, 10-m_t]
            
            #####################################
            result = autocorrelation(input_seq, m_x, m_y, m_t, device=device)
            #####################################
            
            for x in m_x_fill:
                for y in m_y_fill:
                    for t in m_t_fill:
                        R_xxx[t][y][x] = result

end_t = time.time()
print('Average time = ', (end_t-start_t)/(21*144*176))

KeyboardInterrupt: 